In [2]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [3]:
df = spark.read.parquet("gs://dabi-project-bucket/DWH/DIM_weather/rain", inferSchema=True, header=True)
df.createOrReplaceTempView("rain")

In [4]:
df_batch_result = spark.sql("SELECT * FROM rain")

df_batch_result.toPandas()

,stations_id,Datum,Taegliche_Niederschlagshoehe,Niederschlagsform,Tageswert_Schneehoehe
0,3631,1994-01-01,0.5,8.0,0.0
1,3631,1994-01-02,4.6,8.0,0.0
2,3631,1994-01-03,3.0,6.0,0.0
3,3631,1994-01-04,1.5,6.0,0.0
4,3631,1994-01-05,3.5,6.0,0.0
...,...,...,...,...,...
151114,850,2023-12-27,0.3,4.0,0.0
151115,850,2023-12-28,0.6,4.0,0.0
151116,850,2023-12-29,13.0,4.0,0.0
151117,850,2023-12-30,0.0,0.0,NaN


In [7]:
df_batch_result = spark.sql(
    """
    SELECT 
        YEAR(Datum) as Jahr, 
        MONTH(Datum) as Monat,
        AVG(Taegliche_Niederschlagshoehe) as avg_Niederschlagshoehe
    FROM 
        rain 
    GROUP BY 
        Jahr, Monat
    """
)

# In Pandas DataFrame umwandeln
df_batch_result.toPandas()

,Jahr,Monat,avg(Taegliche_Niederschlagshoehe)
0,1997,11,1.440769
1,2022,10,1.837097
2,1995,12,1.778495
3,2005,5,2.766590
4,1998,7,2.516935
...,...,...,...
355,2015,5,2.175914
356,1996,8,2.900993
357,2007,3,2.553456
358,2021,4,1.284762


In [8]:
df_batch_result.write.parquet("gs://dabi-project-bucket/DWH/Analitic_Results/Rain", mode="overwrite")